In [1]:
import tensorflow as tf
import numpy as np

/home/brad/.conda/envs/tensorflow_gpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/brad/.conda/envs/tensorflow_gpu/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [31]:
def euclidean_dsq(X):
    
    ndim, npts = np.shape(X)
    M = np.dot(X.T, X)
    Xsq = np.sum(np.square(X), axis=0)[np.newaxis, :]
    Xii = np.dot(np.ones((npts, 1)), Xsq)
    Xjj = np.dot(Xsq.T, np.ones((1, npts)))
    
    return Xii + Xjj - 2*M

def Dij(X):
    '''
    Compute hyperbolic distances between pairs of points in X
    X = ndim x npts
    '''
    M = euclidean_dsq(X)
    a = 1 - np.sum(np.square(X), axis=0)[np.newaxis,:]
    A = np.outer(a, a)
    C = 1 + 2* np.divide(M, A)
    return np.arccosh(C)

def Aij(X, r, t):
    
    D = Dij(X)
    return np.power(np.exp((D - r)/t) +1, -1)

def d_Dij(X):
    ndim, npts = np.shape(X)
    alpha = 1-np.sum(np.square(X), axis=0)[np.newaxis, :]
    beta = 1-np.sum(np.square(X), axis=0)[np.newaxis, :]
    ab = np.outer(alpha, beta)
    gamma = 1 + 2*np.divide(euclidean_dsq(X), ab)
    
    Xsq = np.sum(np.square(X), axis=0)[np.newaxis, :]
    
    aij = np.dot(np.ones((npts, 1)), alpha)
    Xsqij = np.dot(np.ones((npts, 1)), Xsq)
    Mij = np.dot(X.T, X)
    #Xij = np.dot(np.ones((npts, 1)), X[np.newaxis, :, :])
    Xij = np.einsum('ia, jab->jib', np.ones((npts, 1)), X[:, np.newaxis, :])
    #ai = np.dot(alpha.T, np.ones((1, npts)))
    
    coeff = 4 / (np.multiply(aij, np.sqrt(np.square(gamma) - 1)))
    coeff = np.einsum('ij, jab->iab', np.ones((ndim, 1)), coeff[np.newaxis, :, :])
    
    c2 = np.divide(Xsqij - 2*Mij + 1, np.square(aij.T))
    c2 = np.einsum('ij, jab->iab', np.ones((ndim, 1)), c2[np.newaxis, :, :])
    
    c3 = np.divide(Xij, np.einsum('ij, jab->iab', np.ones((ndim, 1)), (aij.T)[np.newaxis, :, :])
    c4 = np.multiply(c2, Xij.T) - c3
    c5 = np.multiply(coeff, c4)
    
    

In [22]:
def poincare_loss(X, Ahat, r, t):
    ''' X is npts x ndim '''
    
    r = tf.reduce_sum(X*X, 1)
    r = tf.reshape(r, [-1, 1])
    M = r - 2*tf.matmul(X, tf.transpose(X)) + tf.transpose(r)
    a = r*tf.transpose(r)
    C = 1+2*tf.divide(M, a)
    D = tf.acosh(C)
    A = 1 / (tf.exp((D - r)/t) + 1)
    H = -(tf.multiply(Ahat, tf.log(A)) + tf.multiply((1-Ahat), tf.log(1-A)))
    
    
    M = tf.matmul(tf.transpose(X), X)
    Xsq = tf.sum(tf.square(X), axis=0)[tf.newaxis, :]
    Xii
    
        ndim, npts = np.shape(X)
    M = np.dot(X.T, X)
    Xsq = np.sum(np.square(X), axis=0)[np.newaxis, :]
    Xii = np.dot(np.ones((npts, 1)), Xsq)
    Xjj = np.dot(Xsq.T, np.ones((1, npts)))
    
    return Xii + Xjj - 2*M

array([[0.25, 0.  , 0.  , 0.1 ],
       [0.  , 0.15, 0.  , 0.6 ]])

In [28]:
Aij(x, 0, 1)

array([[0.5       , 0.35432859, 0.375     , 0.18652608],
       [0.35432859, 0.5       , 0.425     , 0.24675021],
       [0.375     , 0.425     , 0.5       , 0.19586187],
       [0.18652608, 0.24675021, 0.19586187, 0.5       ]])

In [32]:
d_Dij(x)

ValueError: shapes (4,1) and (1,2,4) not aligned: 1 (dim 1) != 2 (dim 1)